## Seldon V2 Experiment Version Tests


### Test change candidate for a model

We will use three SKlearn Iris classification models to illustrate experiments.

Load both models.

In [6]:
!seldon model load -f ./models/sklearn1.yaml
!seldon model load -f ./models/sklearn2.yaml
!seldon model load -f ./models/sklearn3.yaml

{}
{}
{}


In [7]:
!seldon model status iris -w ModelAvailable
!seldon model status iris2 -w ModelAvailable
!seldon model status iris3 -w ModelAvailable

{}
{}
{}


In [8]:
!seldon model infer iris -i 50 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

map[:iris_1::50]


In [9]:
!seldon model infer iris2 -i 50 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

map[:iris2_1::50]


In [10]:
!seldon model infer iris3 -i 50 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

map[:iris3_1::50]


In [11]:
!cat ./experiments/ab-default-model.yaml 

apiVersion: mlops.seldon.io/v1alpha1
kind: Experiment
metadata:
  name: experiment-sample
spec:
  default: iris
  candidates:
  - name: iris
    weight: 50
  - name: iris2
    weight: 50


In [12]:
!seldon experiment start -f ./experiments/ab-default-model.yaml 

{}


In [13]:
!seldon experiment status experiment-sample -w | jq -M .

{
  "experimentName": "experiment-sample",
  "active": true,
  "candidatesReady": true,
  "mirrorReady": true,
  "statusDescription": "experiment active",
  "kubernetesMeta": {}
}


In [14]:
!seldon model infer iris -i 50 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

map[:iris2_1::26 :iris_1::24]


In [15]:
!cat ./experiments/ab-default-model2.yaml 

apiVersion: mlops.seldon.io/v1alpha1
kind: Experiment
metadata:
  name: experiment-sample
spec:
  default: iris
  candidates:
  - name: iris
    weight: 50
  - name: iris3
    weight: 50


In [16]:
!seldon experiment start -f ./experiments/ab-default-model2.yaml 

{}


In [17]:
!seldon experiment status experiment-sample -w | jq -M .

{
  "experimentName": "experiment-sample",
  "active": true,
  "candidatesReady": true,
  "mirrorReady": true,
  "statusDescription": "experiment active",
  "kubernetesMeta": {}
}


In [18]:
!seldon model infer iris -i 50 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

map[:iris3_1::24 :iris_1::26]


In [19]:
!seldon experiment stop experiment-sample

{}


In [20]:
!seldon model infer iris -i 50 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

map[:iris_1::50]


In [21]:
!seldon model infer iris2 -i 50 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

map[:iris2_1::50]


In [22]:
!seldon model infer iris3 -i 50 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

map[:iris3_1::50]


In [23]:
!seldon model unload iris
!seldon model unload iris2
!seldon model unload iris3

{}
{}
{}


### Test change default model in an experiment

We will use three SKlearn Iris classification models to illustrate experiments.

In [24]:
!seldon model load -f ./models/sklearn1.yaml
!seldon model load -f ./models/sklearn2.yaml
!seldon model load -f ./models/sklearn3.yaml

{}
{}
{}


In [25]:
!seldon model status iris -w ModelAvailable
!seldon model status iris2 -w ModelAvailable
!seldon model status iris3 -w ModelAvailable

{}
{}
{}


In [26]:
!seldon model infer iris -i 50 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

map[:iris_1::50]


In [27]:
!seldon model infer iris2 -i 50 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

map[:iris2_1::50]


In [28]:
!seldon model infer iris3 -i 50 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

map[:iris3_1::50]


In [29]:
!cat ./experiments/ab-default-model.yaml 

apiVersion: mlops.seldon.io/v1alpha1
kind: Experiment
metadata:
  name: experiment-sample
spec:
  default: iris
  candidates:
  - name: iris
    weight: 50
  - name: iris2
    weight: 50


In [30]:
!seldon experiment start -f ./experiments/ab-default-model.yaml 

{}


In [31]:
!seldon experiment status experiment-sample -w | jq -M .

{
  "experimentName": "experiment-sample",
  "active": true,
  "candidatesReady": true,
  "mirrorReady": true,
  "statusDescription": "experiment active",
  "kubernetesMeta": {}
}


Run a set of calls and record which route the traffic took. There should be roughly a 50/50 split.

In [32]:
!seldon model infer iris -i 50 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

map[:iris2_1::28 :iris_1::22]


In [33]:
!cat ./experiments/ab-default-model3.yaml 

apiVersion: mlops.seldon.io/v1alpha1
kind: Experiment
metadata:
  name: experiment-sample
spec:
  default: iris3
  candidates:
  - name: iris3
    weight: 50
  - name: iris2
    weight: 50


In [34]:
!seldon experiment start -f ./experiments/ab-default-model3.yaml 

{}


In [35]:
!seldon experiment status experiment-sample -w | jq -M .

{
  "experimentName": "experiment-sample",
  "active": true,
  "candidatesReady": true,
  "mirrorReady": true,
  "statusDescription": "experiment active",
  "kubernetesMeta": {}
}


In [36]:
!seldon model infer iris -i 50 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

map[:iris_1::50]


In [37]:
!seldon model infer iris3 -i 50 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

map[:iris2_1::24 :iris3_1::26]


In [38]:
!seldon model infer iris2 -i 50 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

map[:iris2_1::50]


In [39]:
!seldon experiment stop experiment-sample

{}


In [40]:
!seldon model infer iris -i 50 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

map[:iris_1::50]


In [41]:
!seldon model infer iris2 -i 50 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

map[:iris2_1::50]


In [42]:
!seldon model infer iris3 -i 50 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

map[:iris3_1::50]


In [43]:
!seldon model unload iris
!seldon model unload iris2
!seldon model unload iris3

{}
{}
{}
